In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import pandas_profiling
from matplotlib import pyplot as plt

In [2]:
data = pd.read_csv('aac_shelter_outcomes.csv')
data.head()

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
0,2 weeks,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male
1,1 year,A666430,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female
2,1 year,A675708,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male
3,9 years,A680386,Dog,Miniature Schnauzer Mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male
4,5 months,A683115,Other,Bat Mix,Brown,2014-01-07T00:00:00,2014-07-07T14:04:00,2014-07-07T14:04:00,NaN,Rabies Risk,Euthanasia,Unknown


In [3]:
data.shape

(78256, 12)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78256 entries, 0 to 78255
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   age_upon_outcome  78248 non-null  object
 1   animal_id         78256 non-null  object
 2   animal_type       78256 non-null  object
 3   breed             78256 non-null  object
 4   color             78256 non-null  object
 5   date_of_birth     78256 non-null  object
 6   datetime          78256 non-null  object
 7   monthyear         78256 non-null  object
 8   name              54370 non-null  object
 9   outcome_subtype   35963 non-null  object
 10  outcome_type      78244 non-null  object
 11  sex_upon_outcome  78254 non-null  object
dtypes: object(12)
memory usage: 7.2+ MB


In [5]:
data.isnull().sum()

age_upon_outcome        8
animal_id               0
animal_type             0
breed                   0
color                   0
date_of_birth           0
datetime                0
monthyear               0
name                23886
outcome_subtype     42293
outcome_type           12
sex_upon_outcome        2
dtype: int64

Посмотрим на признаки: **age_upon_outcome**, **animal_type**, **breed**, **color**, **date_of_birth**, **datetime**, **monthyear**, **sex_upon_outcome**. В **outcome_subtype** больше половины пропусков, поэтому вряд ли он будет полезен.

Из нужных нам данных, неопределённые значения только в признаках: **age_upon_outcome**, **outcome_type** и **sex_upon_outcome**.

In [6]:
# Проверим являются ли даты в колонках datetime и monthyear одинаковыми
len(data[data['datetime'] != data['monthyear']])

0

In [7]:
# Удалим ненужные признаки 
data.drop(['animal_id', 'name', 'outcome_subtype', 'monthyear'], axis=1, inplace=True)
data.head()

,age_upon_outcome,animal_type,breed,color,date_of_birth,datetime,outcome_type,sex_upon_outcome
0,2 weeks,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,Transfer,Intact Male
1,1 year,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,Transfer,Spayed Female
2,1 year,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,Adoption,Neutered Male
3,9 years,Dog,Miniature Schnauzer Mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,Transfer,Neutered Male
4,5 months,Other,Bat Mix,Brown,2014-01-07T00:00:00,2014-07-07T14:04:00,Euthanasia,Unknown


In [8]:
# Посмотрим на уникальные значеничя колонки age_upon_outcome
data['age_upon_outcome'].unique()

array(['2 weeks', '1 year', '9 years', '5 months', '4 months', '3 years',
       '1 month', '3 months', '2 years', '2 months', '4 years', '8 years',
       '3 weeks', '8 months', '12 years', '7 years', '5 years', '6 years',
       '5 days', '10 months', '4 weeks', '10 years', '2 days', '6 months',
       '14 years', '11 months', '15 years', '7 months', '13 years',
       '11 years', '16 years', '9 months', '3 days', '6 days', '4 days',
       '5 weeks', '1 week', '1 day', '1 weeks', '0 years', '17 years',
       '20 years', '18 years', '19 years', '22 years', '25 years', nan],
      dtype=object)

В признаке **age_upon_outcome** присутствуют значения *'0 years'* и NaN. Из этого признака создадим новый - возраст в количестве дней.

In [9]:
# Создадим датасет из признака age_upon_outcome без нулевых значений
age_df = data.loc[~data.age_upon_outcome.isnull() ,['age_upon_outcome']]

In [10]:
age_df[['num', 'unit']] = age_df.age_upon_outcome.str.extract(r'(\d*)(\D*)')
age_df.head()

,age_upon_outcome,num,unit
0,2 weeks,2,weeks
1,1 year,1,year
2,1 year,1,year
3,9 years,9,years
4,5 months,5,months


In [11]:
# Посмотрим как записан возраст
age_df.unit.value_counts()

 years     31112
 months    23436
 year      14355
 weeks      4565
 month      3344
 days        856
 week        427
 day         153
Name: unit, dtype: int64

In [12]:
age_df.loc[age_df.unit.str.contains(r'year'), 'unit_count'] = 365
age_df.loc[age_df.unit.str.contains(r'month'), 'unit_count'] = 30
age_df.loc[age_df.unit.str.contains(r'week'), 'unit_count'] = 7
age_df.loc[age_df.unit.str.contains(r'day'), 'unit_count'] = 1
age_df.head()

,age_upon_outcome,num,unit,unit_count
0,2 weeks,2,weeks,7.0
1,1 year,1,year,365.0
2,1 year,1,year,365.0
3,9 years,9,years,365.0
4,5 months,5,months,30.0


In [13]:
age_df.num = age_df.num.astype(int)

In [14]:
# Создадим новый признак из количества дней
age_df['age'] = age_df.apply(lambda row: row.num * row.unit_count, axis=1)

In [15]:
# Присоединим к датасету признак age
data = data.join(age_df[['age']], how='left')
data.head()

,age_upon_outcome,animal_type,breed,color,date_of_birth,datetime,outcome_type,sex_upon_outcome,age
0,2 weeks,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,Transfer,Intact Male,14.0
1,1 year,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,Transfer,Spayed Female,365.0
2,1 year,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,Adoption,Neutered Male,365.0
3,9 years,Dog,Miniature Schnauzer Mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,Transfer,Neutered Male,3285.0
4,5 months,Other,Bat Mix,Brown,2014-01-07T00:00:00,2014-07-07T14:04:00,Euthanasia,Unknown,150.0


In [16]:
# Посмотрим на неопределённые значения признака age, они соответствуют неопределённым значениям 
# признака age_upon_outcome
data[data.age.isnull()]

,age_upon_outcome,animal_type,breed,color,date_of_birth,datetime,outcome_type,sex_upon_outcome,age
68246,NaN,Dog,Labrador Retriever Mix,Black/White,2013-11-02T00:00:00,2016-11-19T16:35:00,NaN,NaN,NaN
76825,NaN,Dog,Pit Bull Mix,Black/White,2016-12-27T00:00:00,2017-12-30T16:47:00,NaN,Intact Female,NaN
77976,NaN,Bird,Leghorn Mix,White/Red,2017-01-22T00:00:00,2018-01-25T13:23:00,Transfer,Intact Female,NaN
78081,NaN,Dog,Miniature Poodle Mix,Black,2011-01-29T00:00:00,2018-01-29T15:49:00,Euthanasia,Neutered Male,NaN
78114,NaN,Cat,Domestic Shorthair Mix,Lynx Point,2017-01-29T00:00:00,2018-01-29T18:08:00,Euthanasia,Intact Male,NaN
78162,NaN,Dog,Maltese Mix,Buff,2017-01-29T00:00:00,2018-01-31T08:14:00,Transfer,Intact Male,NaN
78208,NaN,Dog,Beagle/Catahoula,Tan/White,2010-02-01T00:00:00,2018-02-01T09:21:00,Euthanasia,Intact Male,NaN
78253,NaN,Other,Bat Mix,Brown,2017-02-01T00:00:00,2018-02-01T18:08:00,Euthanasia,Unknown,NaN


Для неопределенных значений возраста животного, вычислим возраст, посчитав из признаков **datetime** и **date_of_birth**.

In [17]:
# Переведём признаки datetime и date_of_birth в тип данных data
data['datetime'] = data['datetime'].astype('datetime64[D]')
data['date_of_birth'] = data['date_of_birth'].astype('datetime64[D]')

In [18]:
# Посчитаем возраст из количества дней из признаков datetime и date_of_birth и переведём в численный вид
# data.loc[data.age.isnull(), 'age'] = data.age.fillna(data[data.age.isnull()]
#                                                      ['datetime'].sub(data[data.age.isnull()]['date_of_birth'])
#                                                      .dt.days)
data.age.fillna(data[data.age.isnull()]['datetime'].sub(data[data.age.isnull()]['date_of_birth'])
                .dt.days, inplace=True)

In [19]:
# Проверим значения признака age, которые были неопределёнными
data[data.age_upon_outcome.isnull()]

,age_upon_outcome,animal_type,breed,color,date_of_birth,datetime,outcome_type,sex_upon_outcome,age
68246,NaN,Dog,Labrador Retriever Mix,Black/White,2013-11-02,2016-11-19,NaN,NaN,1113.0
76825,NaN,Dog,Pit Bull Mix,Black/White,2016-12-27,2017-12-30,NaN,Intact Female,368.0
77976,NaN,Bird,Leghorn Mix,White/Red,2017-01-22,2018-01-25,Transfer,Intact Female,368.0
78081,NaN,Dog,Miniature Poodle Mix,Black,2011-01-29,2018-01-29,Euthanasia,Neutered Male,2557.0
78114,NaN,Cat,Domestic Shorthair Mix,Lynx Point,2017-01-29,2018-01-29,Euthanasia,Intact Male,365.0
78162,NaN,Dog,Maltese Mix,Buff,2017-01-29,2018-01-31,Transfer,Intact Male,367.0
78208,NaN,Dog,Beagle/Catahoula,Tan/White,2010-02-01,2018-02-01,Euthanasia,Intact Male,2922.0
78253,NaN,Other,Bat Mix,Brown,2017-02-01,2018-02-01,Euthanasia,Unknown,365.0


In [20]:
# Переведём значения признака age в целочисленный тип
data['age'] = data['age'].astype(int)

In [21]:
# Посчитаем нулевые значения признака age
len(data[data.age == 0])

95

In [22]:
# Удалим нулевые значения признака age, а также неопределённые значения признака outcome_type
data = data.loc[(data['age'] != 0) & (~data.outcome_type.isna())].reset_index(drop=True)

In [23]:
data.shape

(78150, 9)

In [24]:
# Посмотрим на виды животных
data['animal_type'].value_counts()

Dog          44208
Cat          29369
Other         4230
Bird           334
Livestock        9
Name: animal_type, dtype: int64

In [25]:
# Посмотрим на значения признака breed животных типа Other
data[(data.animal_type == 'Other')]['breed'].unique()

array(['Bat Mix', 'Squirrel Mix', 'Bat', 'Raccoon Mix', 'Armadillo Mix',
       'Rat', 'Hamster Mix', 'Raccoon', 'Rabbit Sh Mix', 'Fox Mix',
       'American Sable', 'Opossum Mix', 'Guinea Pig Mix', 'Rex Mix',
       'Rat Mix', 'Rabbit Sh', 'Skunk', 'Tortoise Mix', 'Skunk Mix',
       'Californian Mix', 'Lionhead Mix', 'Opossum', 'Cold Water',
       'Squirrel', 'Guinea Pig', 'Ferret Mix', 'Ferret', 'Rabbit Lh Mix',
       'New Zealand Wht Mix', 'Coyote Mix', 'Lop-Mini Mix', 'Fox',
       'Cottontail Mix', 'Jersey Wooly Mix', 'Turtle Mix', 'American Mix',
       'Lizard Mix', 'Dutch/Rabbit Sh', 'English Spot Mix', 'Armadillo',
       'Lop-Holland Mix', 'Rabbit Sh/Lop-Mini', 'Hamster', 'Lionhead',
       'Dutch Mix', 'Californian', 'Silver Mix', 'Lop-English Mix',
       'Sugar Glider', 'Snake', 'Snake Mix', 'Hotot Mix', 'Rabbit Lh',
       'Mouse Mix', 'Chinchilla-Stnd Mix', 'Checkered Giant Mix',
       'Coyote', 'Netherlnd Dwarf Mix', 'Havana Mix', 'Lop-Mini',
       'Otter Mix', 'Bo

In [26]:
# Посмотрим на значения признака breed птиц
data[(data.animal_type == 'Bird')]['breed'].unique()

array(['Chicken', 'Parakeet Mix', 'Bantam Mix', 'Duck', 'Lark Mix',
       'Chicken Mix', 'Sparrow Mix', 'Parrot Mix', 'Duck Mix', 'Dove Mix',
       'Nuthatch Mix', 'Cockatiel', 'Pigeon Mix', 'Catbird Mix', 'Finch',
       'Cockatiel Mix', 'Rhode Island Mix', 'Lovebird Mix', 'Grackle Mix',
       'Quaker Mix', 'Silkie Mix', 'Parakeet', 'Guinea Mix', 'Turkey Mix',
       'Muscovy', 'Parrot', 'Dove', 'Peafowl Mix', 'Cockatoo Mix', 'Hawk',
       'Cockatoo', 'Mockingbird Mix', 'Vulture Mix', 'Owl Mix', 'Grackle',
       'Pigeon', 'Bluebird Mix', 'Bunting', 'Muscovy Mix', 'Heron Mix',
       'Conure', 'Crow Mix', 'Heron', 'Peafowl', 'Budgerigar', 'Quaker',
       'Song Bird Mix', 'African Mix', 'Hawk Mix', 'Conure Mix',
       'Wren Mix', 'Mockingbird', 'Goose Mix', 'Barred Rock Mix',
       'Sparrow', 'Owl', 'Leghorn Mix', 'Pheasant Mix'], dtype=object)

Создадим новый признак для неодомашненных животных и птиц. Животных отберём только для типа *Other* , т.к. *Livestock* это домашний скот. 

In [27]:
# Создадим кортеж из диких животных и птиц
not_domestic = ('Lizard', 'Armadillo', 'Coyote', 'Fox', 'Squirrel', 'Skunk', 'Opossum', 'Raccoon', 'Bat', 'American Sable', 
            'Tarantula', 'Snake', 'Deer', 'Otter', 'Frog', 'Hedgehog', 'Lark', 'Sparrow', 'Dove', 'Nuthatch', 'Pigeon', 
            'Catbird', 'Finch', 'Grackle', 'Peafowl', 'Hawk', 'Mockingbird', 'Vulture', 'Owl', 'Bluebird', 'Bunting', 
            'Heron', 'Crow', 'Song Bird', 'Wren')

Животных *Bird* и *Livestock* не много. Для уменьшения размерности присвоим им категорию *Other*.

In [28]:
# Значения Bird и Livestock поменяем на Other
data['animal_type'] = data['animal_type'].replace('Bird', 'Other').replace('Livestock', 'Other')
data['animal_type'].value_counts()

Dog      44208
Cat      29369
Other     4573
Name: animal_type, dtype: int64

In [29]:
# В новом признаке неодомашненным животным присвоим значение 1, остальным 0
data.loc[(data.animal_type == 'Other') & (data.breed.str.startswith(not_domestic)), 'not_domestic'] = 1
data.not_domestic.fillna(0, inplace=True)
data.head()

,age_upon_outcome,animal_type,breed,color,date_of_birth,datetime,outcome_type,sex_upon_outcome,age,not_domestic
0,2 weeks,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07,2014-07-22,Transfer,Intact Male,14,0.0
1,1 year,Dog,Beagle Mix,White/Brown,2012-11-06,2013-11-07,Transfer,Spayed Female,365,0.0
2,1 year,Dog,Pit Bull,Blue/White,2013-03-31,2014-06-03,Adoption,Neutered Male,365,0.0
3,9 years,Dog,Miniature Schnauzer Mix,White,2005-06-02,2014-06-15,Transfer,Neutered Male,3285,0.0
4,5 months,Other,Bat Mix,Brown,2014-01-07,2014-07-07,Euthanasia,Unknown,150,1.0


In [30]:
data.not_domestic.value_counts()

0.0    74652
1.0     3498
Name: not_domestic, dtype: int64

In [31]:
# Посмотрим на породы кошек
cat_breed = data[data['animal_type'] == 'Cat'][['breed']]
cat_breed['breed'].value_counts()

Domestic Shorthair Mix                  23285
Domestic Medium Hair Mix                 2323
Domestic Longhair Mix                    1227
Siamese Mix                               998
Domestic Shorthair                        385
                                        ...  
Manx                                        1
Domestic Shorthair/British Shorthair        1
American Wirehair Mix                       1
Angora/Persian                              1
Siamese/Angora                              1
Name: breed, Length: 79, dtype: int64

Разделим породы кошек на короткошёрстных, среднешёрстных и длинношёрстных.

In [32]:
# Выделим породы, в названиях которых не содержатся 'Shorthair', 'Medium Hair' и 'Longhair'
cat_breed_unver = cat_breed[~((cat_breed['breed'].str.contains('Longhair', case=False))
                              | (cat_breed['breed'].str.contains('Shorthair', case=False))
                              | (cat_breed['breed'].str.contains('Medium Hair', case=False)))][['breed']]
cat_breed_unver['breed'].unique()

array(['Siamese', 'Russian Blue Mix', 'Siamese Mix', 'Manx Mix',
       'Ragdoll Mix', 'Snowshoe', 'Angora Mix', 'Himalayan Mix',
       'Snowshoe Mix', 'Japanese Bobtail Mix', 'Maine Coon Mix',
       'Russian Blue', 'Devon Rex Mix', 'Balinese Mix', 'Angora/Persian',
       'Devon Rex', 'Ragdoll', 'Persian Mix', 'Cymric Mix',
       'Tonkinese Mix', 'Maine Coon', 'Siamese/Angora', 'Persian',
       'Burmese', 'Sphynx', 'Bengal Mix', 'Bombay Mix', 'Himalayan',
       'Abyssinian Mix', 'Norwegian Forest Cat Mix', 'Bengal',
       'Snowshoe/Ragdoll', 'Manx/Siamese', 'Turkish Van Mix',
       'Cornish Rex Mix', 'Birman Mix', 'Siamese/Japanese Bobtail',
       'Havana Brown Mix', 'American Wirehair Mix', 'Scottish Fold Mix',
       'Oriental Sh Mix', 'Chartreux Mix', 'Turkish Angora Mix',
       'Javanese Mix', 'Manx', 'Ocicat Mix'], dtype=object)

In [33]:
# Создадим списки из пород, относительно длины их шерсти
short = ['Siamese', 'Siamese Mix', 'Siamese/Japanese Bobtail', 'Siamese/Angora', 'Russian Blue Mix', 'Russian Blue',
         'Manx', 'Manx Mix', 'Manx/Siamese', 'Snowshoe', 'Snowshoe Mix', 'Snowshoe/Ragdoll', 'Japanese Bobtail Mix',
         'Devon Rex Mix', 'Devon Rex', 'Tonkinese Mix', 'Burmese', 'Sphynx', 'Bengal Mix', 'Bombay Mix',
         'Abyssinian Mix', 'Bengal', 'Cornish Rex Mix', 'Havana Brown Mix', 'Scottish Fold Mix', 'Oriental Sh Mix',
         'Chartreux Mix', 'Ocicat Mix']

medium = ['Ragdoll Mix', 'Ragdoll', 'Angora Mix', 'Turkish Angora Mix', 'Balinese Mix', 'Turkish Van Mix',
          'Birman Mix', 'American Wirehair Mix', 'Javanese Mix']

long = ['Himalayan', 'Himalayan Mix','Maine Coon', 'Maine Coon Mix','Persian', 'Persian Mix', 'Angora/Persian',
        'Cymric Mix', 'Norwegian Forest Cat Mix']

In [34]:
def hair_type(row):
    """Возвращает 'Shorthair', 'Medium Hair' или 'Longhair' в зависимости от названия породы"""
    if 'Shorthair' in row.breed or row.breed in short:
        return 'shorthair'
    elif 'Medium Hair' in row.breed or row.breed in medium:
        return 'medium hair'
    elif 'Longhair' in row.breed or row.breed in long:
        return 'longhair'

In [35]:
# Создадим признак cat_hair_type - длина шерсти кошки
data['cat_hair_type'] = data[data['animal_type'] == 'Cat'].apply(hair_type, axis=1)
data.head()

,age_upon_outcome,animal_type,breed,color,date_of_birth,datetime,outcome_type,sex_upon_outcome,age,not_domestic,cat_hair_type
0,2 weeks,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07,2014-07-22,Transfer,Intact Male,14,0.0,shorthair
1,1 year,Dog,Beagle Mix,White/Brown,2012-11-06,2013-11-07,Transfer,Spayed Female,365,0.0,NaN
2,1 year,Dog,Pit Bull,Blue/White,2013-03-31,2014-06-03,Adoption,Neutered Male,365,0.0,NaN
3,9 years,Dog,Miniature Schnauzer Mix,White,2005-06-02,2014-06-15,Transfer,Neutered Male,3285,0.0,NaN
4,5 months,Other,Bat Mix,Brown,2014-01-07,2014-07-07,Euthanasia,Unknown,150,1.0,NaN


In [36]:
# Проверим наличие значения признака cat_hair_type у всех кошек
data['cat_hair_type'].isnull().sum() - len(data[data['animal_type'] != 'Cat'])

0

In [37]:
# Неопределённым значениям признака cat_hair_type присвоим значение 'not cat'
data.cat_hair_type.fillna(value='hair_undef', inplace=True)

In [38]:
# Посмотрим на породы собак. Создадим новый датасет dog_breed_df
dog_breed_df = data[data['animal_type'] == 'Dog'][['breed']]
dog_breed_df['breed'].tolist()

['Beagle Mix',
 'Pit Bull',
 'Miniature Schnauzer Mix',
 'Leonberger Mix',
 'Chihuahua Shorthair Mix',
 'Papillon/Border Collie',
 'Chihuahua Shorthair/Pomeranian',
 'Miniature Schnauzer/Miniature Poodle',
 'Labrador Retriever Mix',
 'Rat Terrier Mix',
 'Pit Bull Mix',
 'German Shepherd/Labrador Retriever',
 'Beagle Mix',
 'Mastiff Mix',
 'Plott Hound Mix',
 'Labrador Retriever Mix',
 'Chihuahua Shorthair Mix',
 'Labrador Retriever Mix',
 'Labrador Retriever Mix',
 'Chihuahua Shorthair Mix',
 'Chihuahua Shorthair Mix',
 'Miniature Pinscher Mix',
 'Cardigan Welsh Corgi/English Setter',
 'Border Collie Mix',
 'Pit Bull Mix',
 'Boxer Mix',
 'Parson Russell Terrier Mix',
 'Pit Bull Mix',
 'Miniature Poodle Mix',
 'Pit Bull Mix',
 'Pit Bull Mix',
 'Miniature Schnauzer Mix',
 'Chihuahua Shorthair Mix',
 'Australian Shepherd/Australian Cattle Dog',
 'Labrador Retriever Mix',
 'Basenji/Chihuahua Longhair',
 'Chihuahua Shorthair Mix',
 'Pit Bull Mix',
 'Labrador Retriever',
 'Siberian Husky/Ger

Создадим новый признак относительно породы - размер собаки: маленький, средний или большой. Будем считать, что если собака смеси двух пород, то на размер собаки влияет порода, стоящая сначала.

In [39]:
def dog_breed_def(row):
    """Возвращает название породы, стоящей сначала"""
    if 'Mix' in row.breed:
        return re.sub(r'(\s*)Mix', '', row.breed)
    elif '/' in row.breed:
        return row.breed.split('/')[0]
    else:
        return row.breed

In [40]:
# В датасете dog_breed_df создадим новый признак, состоящий из названия одной породы
dog_breed_df['dog_breed'] = dog_breed_df.apply(dog_breed_def, axis=1)

In [41]:
# Проверим отсутствие неопределённых значений
dog_breed_df['dog_breed'].isnull().sum()

0

In [42]:
# Посмотрим на уникальные названия пород собак в датасете
set(dog_breed_df['dog_breed'].tolist())

{'Affenpinscher',
 'Afghan Hound',
 'Airedale Terrier',
 'Akbash',
 'Akita',
 'Alaskan Husky',
 'Alaskan Malamute',
 'American Bulldog',
 'American Eskimo',
 'American Foxhound',
 'American Pit Bull Terrier',
 'American Staffordshire Terrier',
 'Anatol Shepherd',
 'Australian Cattle Dog',
 'Australian Kelpie',
 'Australian Shepherd',
 'Australian Terrier',
 'Basenji',
 'Basset Hound',
 'Beagle',
 'Bearded Collie',
 'Beauceron',
 'Bedlington Terr',
 'Belgian Malinois',
 'Belgian Sheepdog',
 'Belgian Tervuren',
 'Bernese Mountain Dog',
 'Bichon Frise',
 'Black',
 'Black Mouth Cur',
 'Black/Tan Hound',
 'Bloodhound',
 'Blue Lacy',
 'Bluetick Hound',
 'Boerboel',
 'Border Collie',
 'Border Terrier',
 'Boston Terrier',
 'Bouv Flandres',
 'Boxer',
 'Boykin Span',
 'Brittany',
 'Bruss Griffon',
 'Bull Terrier',
 'Bull Terrier Miniature',
 'Bulldog',
 'Bullmastiff',
 'Cairn Terrier',
 'Canaan Dog',
 'Cane Corso',
 'Cardigan Welsh Corgi',
 'Carolina Dog',
 'Catahoula',
 'Cavalier Span',
 'Chesa

In [43]:
# Создадим кортежи ключевых слов в названии пород собак по их размерам
small_size = ('Miniature', 'Bull Terrier Miniature', 'Chin', 'Japanese', 'Yorkshire', 'Toy', 'Eng Toy', 'Chihuahua',
              'Norwich', 'Norfolk', 'Cairn', 'Skye', 'Border Terrier', 'Jack Russell', 'Boston', 'Australian Terrier',
              'Silky Terrier', 'Rat', 'Tibetan Spaniel', 'Dachshund', 'Havanese', 'Pomeranian', 'Lhasa Apso',
              'Cavalier Span', 'Coton De Tulear', 'Bruss Griffon', 'Dandie Dinmont', 'Maltese', 'Papillon',
              'Pekingese', 'Shih Tzu')

medium_size = ('Standard', 'Australian Kelpie', 'Manchester', 'Pit', 'American Pit', 'Irish Terrier', 'Wire Hair Fox',
                'Smooth Fox', 'Soft', 'Lakeland', 'Parson Russell', 'Scottish Terrier', 'Tibetan Terrier',
                'Staffordshire', 'American Staffordshire', 'Cocker', 'English Cocker', 'Welsh', 'Field',
                'English Springer', 'Cardigan', 'Pembroke', 'French Bulldog', 'Beagle', 'Australian Cattle',
                'Glen Of Imaal', 'German Pinscher', 'Italian Greyhound', 'Basset', 'Shetland', 'Finnish',
                'Affenpinscher', 'Lowchen', 'American Eskimo', 'Basenji', 'Bedlington', 'Bichon Frise', 'Black Mouth',
                'Boykin', 'Feist', 'Mexican Hairless', 'Patterdale', 'Pbgv', 'Podengo Pequeno', 'Pug',
                'Queensland Heeler', 'Schipperke', 'Sealyham', 'Shiba Inu', 'Sussex', 'Swedish', 'West Highland')

large_size = ('Anatol', 'Australian Shepherd', 'English Shepherd', 'German Shepherd', 'Dutch', 'Schnauzer',
               'English Setter', 'Irish Setter', 'Gordon', 'Irish Wolfhound', 'St. Bernard', 'Collie', 'Bearded',
               'Border Collie', 'Bull Terrier', 'Airedale', 'Bulldog', 'American Bulldog', 'English Bulldog',
               'Old English', 'French', 'Boxer', 'Bullmastiff', 'Chow Chow', 'Dogo', 'Canaan', 'Carolina', 'Bernese',
               'Port', 'Greater', 'Dogue', 'Dalmatian', 'American Foxhound', 'English Foxhound', 'Pointer', 'Grand', 
               'English Pointer', 'German Shorthair', 'German Wirehaired', 'Labrador', 'Flat', 'Golden', 'Nova',
               'Spinone', 'Greyhound', 'Italian', 'Harrier', 'Hovawart', 'Mastiff', 'Spanish', 'Neapolitan',
               'Newfoundland', 'Norwegian', 'Picardy', 'Alaskan', 'Siberian', 'Wirehaired Pointing Griffon', 'Great',
               'English Coonhound', 'Bloodhound', 'Otterhound', 'Doberman', 'Belgian', 'Leonberger', 'Landseer',
               'Cane Corso', 'Catahoula', 'Chesa Bay', 'Akbash', 'Akita', 'Beauceron', 'Blue Lacy', 'Boerboel',
               'Bouv Flandres', 'Brittany', 'Entlebucher', 'Jindo', 'Keeshond', 'Kuvasz', 'Presa Canario',
               'Rhod Ridgeback', 'Rottweiler', 'Saluki', 'Samoyed', 'Treeing', 'Vizsla', 'Weimaraner', 'Whippet',
               'Black', 'Swiss', 'Ibizan', 'Plott', 'Redbone', 'Bluetick', 'Afghan', 'Pharaoh', )

In [44]:
# Создадим новый признак dog_size в датасете dog_breed_df
# Для определения размера собаки берётся название породы, стоящее сначала
dog_breed_df.loc[dog_breed_df['dog_breed'].str.startswith(large_size), 'dog_size'] = 'large'
dog_breed_df.loc[dog_breed_df['dog_breed'].str.startswith(medium_size), 'dog_size'] = 'medium'
dog_breed_df.loc[dog_breed_df['dog_breed'].str.startswith(small_size), 'dog_size'] = 'small'

In [45]:
# Проверим наличие неопределённых значений
dog_breed_df[dog_breed_df.dog_size.isnull()]

,breed,dog_breed,dog_size
24236,Unknown Mix,Unknown,NaN


In [46]:
# Посмотрим на запись собаки неопределённой породы
data.iloc[24239, :]

age_upon_outcome                 11 months
animal_type                            Cat
breed               Domestic Shorthair Mix
color                    Brown Tabby/White
date_of_birth          2014-02-26 00:00:00
datetime               2015-02-17 00:00:00
outcome_type                      Transfer
sex_upon_outcome                   Unknown
age                                    330
not_domestic                             0
cat_hair_type                    shorthair
Name: 24239, dtype: object

In [47]:
# Удалим значение неизвестной породы
data = data.loc[data['breed'] != 'Unknown Mix'].reset_index(drop=True)

In [48]:
# Соединим датафрейм data с датафреймом признака dog_size
data = data.join(dog_breed_df[['dog_size']], how='left')

In [49]:
# Неопределённым значениям признака dog_size присвоим значение 'not dog'
data.dog_size.fillna(value='size_undef', inplace=True)
data.head()

,age_upon_outcome,animal_type,breed,color,date_of_birth,datetime,outcome_type,sex_upon_outcome,age,not_domestic,cat_hair_type,dog_size
0,2 weeks,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07,2014-07-22,Transfer,Intact Male,14,0.0,shorthair,size_undef
1,1 year,Dog,Beagle Mix,White/Brown,2012-11-06,2013-11-07,Transfer,Spayed Female,365,0.0,hair_undef,medium
2,1 year,Dog,Pit Bull,Blue/White,2013-03-31,2014-06-03,Adoption,Neutered Male,365,0.0,hair_undef,medium
3,9 years,Dog,Miniature Schnauzer Mix,White,2005-06-02,2014-06-15,Transfer,Neutered Male,3285,0.0,hair_undef,small
4,5 months,Other,Bat Mix,Brown,2014-01-07,2014-07-07,Euthanasia,Unknown,150,1.0,hair_undef,size_undef


In [50]:
# Удалим ненужные признаки
data.drop(['age_upon_outcome', 'breed', 'date_of_birth', 'datetime'], axis=1, inplace=True)
data.head()

,animal_type,color,outcome_type,sex_upon_outcome,age,not_domestic,cat_hair_type,dog_size
0,Cat,Orange Tabby,Transfer,Intact Male,14,0.0,shorthair,size_undef
1,Dog,White/Brown,Transfer,Spayed Female,365,0.0,hair_undef,medium
2,Dog,Blue/White,Adoption,Neutered Male,365,0.0,hair_undef,medium
3,Dog,White,Transfer,Neutered Male,3285,0.0,hair_undef,small
4,Other,Brown,Euthanasia,Unknown,150,1.0,hair_undef,size_undef


Посмотрим на уникальные значения признака **color**

In [51]:
data['color'].unique()

array(['Orange Tabby', 'White/Brown', 'Blue/White', 'White', 'Brown',
       'Brown/White', 'Tan', 'Blue Tabby/White', 'White/Black',
       'Black/White', 'Black', 'Brown Brindle/White', 'Brown Tabby',
       'Brown Tabby/White', 'Black Tabby', 'Tan/White', 'Brown Brindle',
       'Brown Brindle/Black', 'Yellow', 'Blue/Tan', 'Seal Point',
       'Torbie', 'Buff', 'Red/White', 'Black/Brown', 'Blue',
       'Cream Tabby/White', 'Calico', 'Brown Merle/White', 'Chocolate',
       'White/Brown Brindle', 'Fawn', 'Blue Merle/White', 'Brown/Black',
       'Sable', 'Cream', 'White/Tan', 'Black/Gray', 'Black/Tan',
       'Blue Tabby', 'Gray', 'White/Tricolor', 'Fawn/White', 'Red',
       'Red/Brown', 'Gray/Brown', 'Tricolor', 'Orange Tabby/White',
       'White/Blue', 'Red/Cream', 'Tricolor/Brown', 'Silver Tabby',
       'Flame Point', 'Red Tick/Brown Merle', 'White/Orange Tabby',
       'Chocolate/White', 'Black/Blue Merle', 'Tortie', 'Red Merle/White',
       'White/Buff', 'Cream Tabby', 'Apr

Используем этот признак двумя способами.  
  
  
1-й способ  
Создадим новый признак из трёх типов окраса животных. К 1-му типу col_1 отнесём всех животных с однотонным окрасом. Однотонным окрасом будем считать окрасы, которые состоят из одного цвета и из названий окрасов, близких к однотонным. Ко 2-му типу col_2 отнесём пёстрые окрасы. Такими окрасами будем считать те окрасы, в названиях которых присутствуют названия полосатых, пятнистых, 'черепашьих' и т.д. окрасов. А также смеси этих окрасов с первым типом. Названия окрасов 2-го типа используются чаще для кошек и собак. К 3-му типу отнесём орасы состоящие из нескольких цветов в названии и названия окрасов, которые больше используются для кошек и собак и отличаются от пёстрых большими площадями окраса разными цветами, а также смеси этих окрасов с первым и вторым типом. Таким образом в смеси окрасов из разных типов, по степени влияния будем считать по убыванию от 3-го типа к 1-му.  
  
  
2-й способ  

С помощью CountVectorizer создадим отдельный признак для каждого цвета и вида окраса.

In [52]:
# Применим сначала 2-й способ 
# Создадим массив отдельных названий цветов и окрасов
color_array = np.array([])
for el in data['color'].unique():
    color_array = np.append(color_array, re.split('\/| ', el))

color_array = np.unique(color_array)

In [53]:
color_array.shape

(37,)

In [54]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
count_vect.fit_transform(color_array).toarray()

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]], dtype=int64)

In [55]:
color_columns = count_vect.get_feature_names()
color_mat = count_vect.transform(data.color).toarray()

In [56]:
# color_mat = count_vect.transform(data.color)
#color_df =  pd.DataFrame.sparse.from_spmatrix(color_mat, columns=color_columns)

In [57]:
# Создадим датасет всех цветов и окрасов, встречающихся у животных
color_df = pd.DataFrame(color_mat, columns=color_columns)
color_df.head()

,agouti,apricot,black,blue,brindle,brown,buff,calico,chocolate,cream,...,smoke,tabby,tan,tick,tiger,torbie,tortie,tricolor,white,yellow
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
# Применим 1-й способ
# Посмотрим на уникальные названия цветов и окрасов
color_array

array(['Agouti', 'Apricot', 'Black', 'Blue', 'Brindle', 'Brown', 'Buff',
       'Calico', 'Chocolate', 'Cream', 'Fawn', 'Flame', 'Gold', 'Gray',
       'Green', 'Lilac', 'Liver', 'Lynx', 'Merle', 'Orange', 'Pink',
       'Point', 'Red', 'Ruddy', 'Sable', 'Seal', 'Silver', 'Smoke',
       'Tabby', 'Tan', 'Tick', 'Tiger', 'Torbie', 'Tortie', 'Tricolor',
       'White', 'Yellow'], dtype='<U32')

In [61]:
# Создадим кортеж цветов и кортежи названий окрасов, по 3 типам
colors = ('Apricot', 'Black', 'Blue', 'Brown', 'Buff', 'Chocolate', 'Cream', 'Gold', 'Gray', 'Green', 'Lilac', 'Liver',
          'Orange', 'Pink', 'Red', 'Ruddy', 'Silver', 'White', 'Yellow')

col_1 = ('Smoke', 'Tan', 'Point', 'Fawn')

col_2 = ('Tabby', 'Brindle', 'Tiger', 'Lynx', 'Torbie', 'Tortie')

col_3 = ('Multicolor', 'Tick', 'Agouti', 'Merle', 'Calico', 'Tricolor', 'Sable')

In [62]:
# В признаке присутствует только название цвета
data.loc[(data.color.str.startswith(colors)) & (data.color.str.endswith(colors)), 'color_type'] = 'col_3'
# Признак состоит только из одного цвета
data.loc[(data.color.str.startswith(colors)) & (~data.color.str.contains(r'/')), 'color_type'] = 'col_1'
# В признаке присутствует название однотонного окраса
data.loc[data.color.str.contains(r'Smoke|Tan|Point|Fawn'), 'color_type'] = 'col_1'
# В признаке присутствует название пёстрого окраса
data.loc[data.color.str.contains(r'Tabby|Brindle|Tiger|Lynx|Torbie|Tortie'), 'color_type'] = 'col_2'
# В признаке присутствует название окраса, отличающегося от пёстрого, большими площадями окраса разными цветами
data.loc[data.color.str.contains(r'Multicolor|Tick|Agouti|Merle|Calico|Tricolor|Sable'), 'color_type'] = 'col_3'

In [63]:
data.head()

,animal_type,color,outcome_type,sex_upon_outcome,age,not_domestic,cat_hair_type,dog_size,color_type
0,Cat,Orange Tabby,Transfer,Intact Male,14,0.0,shorthair,size_undef,col_2
1,Dog,White/Brown,Transfer,Spayed Female,365,0.0,hair_undef,medium,col_3
2,Dog,Blue/White,Adoption,Neutered Male,365,0.0,hair_undef,medium,col_3
3,Dog,White,Transfer,Neutered Male,3285,0.0,hair_undef,small,col_1
4,Other,Brown,Euthanasia,Unknown,150,1.0,hair_undef,size_undef,col_1


In [64]:
# Проверим отсутствие неопределённых значений
data['color_type'].isnull().sum()

0

In [65]:
# Посмотрим на уникальные значения признака sex_upon_outcome
data['sex_upon_outcome'].value_counts()

Neutered Male    27781
Spayed Female    25201
Intact Male       9533
Intact Female     9126
Unknown           6507
Name: sex_upon_outcome, dtype: int64

In [66]:
data[(data.outcome_type == 'Adoption') |
     (data.outcome_type == 'Transfer')].groupby(['outcome_type', 'sex_upon_outcome']).count()

animal_type  color    age  not_domestic  \
outcome_type sex_upon_outcome                                            
Adoption     Intact Female             763    763    763           763   
             Intact Male               644    644    644           644   
             Neutered Male           15905  15905  15905         15905   
             Spayed Female           15727  15727  15727         15727   
             Unknown                    70     70     70            70   
Transfer     Intact Female            6099   6099   6099          6099   
             Intact Male              5639   5639   5639          5639   
             Neutered Male            4826   4826   4826          4826   
             Spayed Female            4220   4220   4220          4220   
             Unknown                  2638   2638   2638          2638   

                               cat_hair_type  dog_size  color_type  
outcome_type sex_upon_outcome                                       
Adoption     Intact Female               763       763         763  
             Intact Male                 644       644         644  
             Neutered Male             15905     15905       15905  
             Spayed Female             15727     15727       15727  
             Unknown                      70        70          70  
Transfer     Intact Female              6099      6099        6099  
             Intact Male                5639      5639        5639  
             Neutered Male              4826      4826        4826  
             Spayed Female              4220      4220        4220  
             Unknown                    2638      2638        2638

На значения *Adoption* и *Transfer* целевого признака **outcome_type** существенно влияет способность животного к репродукции.  
Общее количество животных: не репродуктивных/репродуктивных - 52982/18659 *(разница в 2.8 раз)*  
Животных,которых приютили: не репродуктивных/репродуктивных - 31632/1407 *(разница в 22.4 раза)*  
Животных, которых перевели: не репродуктивных/репродуктивных - 9046/11738 *(разница в 0.77 раз)*  
  
  
Половой признак влияет очень слабо.  
Общее количество животных: муж.пол/жен.пол - 37314/34327 *(разница в 1.08 раз)*  
Животных,которых приютили: муж.пол/жен.пол - 16549/16490 *(разница в 1.00 раза)*  
Животных, которых перевели: муж.пол/жен.пол - 10465/10319 *(разница в 1.01 раз)*  
  
  
Значений *Unknown* полового признака мало для значений *Adoption* и много для *Transfer*.  
Разделим признак **sex_upon_outcome** на репродуктивных, не репродуктивных и на неизвестных и создадим новый признак.

In [67]:
# Посмотрим на неопределённые значения
data[data.sex_upon_outcome.isnull()]

,animal_type,color,outcome_type,sex_upon_outcome,age,not_domestic,cat_hair_type,dog_size,color_type
16271,Dog,Brown Merle,Return to Owner,NaN,2555,0.0,hair_undef,small,col_3


In [68]:
# Заменим значение NaN на Unknown
data.sex_upon_outcome.fillna('Unknown', inplace=True)

In [69]:
data.loc[data.sex_upon_outcome.str.startswith('Intact'), 'gender'] = 'Intact'
data.loc[data.sex_upon_outcome.str.startswith(('Neutered', 'Spayed')), 'gender'] = 'Neutered/Spayed'
data.loc[data.sex_upon_outcome.str.startswith('Unknown'), 'gender'] = 'Unknown'

In [70]:
data.gender.value_counts()

Neutered/Spayed    52982
Intact             18659
Unknown             6508
Name: gender, dtype: int64

In [71]:
# Удалим ненужные признаки 
data.drop(['color', 'sex_upon_outcome'], axis=1, inplace=True)
data.head()

,animal_type,outcome_type,age,not_domestic,cat_hair_type,dog_size,color_type,gender
0,Cat,Transfer,14,0.0,shorthair,size_undef,col_2,Intact
1,Dog,Transfer,365,0.0,hair_undef,medium,col_3,Neutered/Spayed
2,Dog,Adoption,365,0.0,hair_undef,medium,col_3,Neutered/Spayed
3,Dog,Transfer,3285,0.0,hair_undef,small,col_1,Neutered/Spayed
4,Other,Euthanasia,150,1.0,hair_undef,size_undef,col_1,Unknown


In [72]:
# Разделим численные и категориальные признаки
number_data = data[['age']]
cat_data = data.drop(['age', 'not_domestic'], axis=1)

In [73]:
# Нормируем численный признак
from sklearn.preprocessing import MinMaxScaler

min_max = MinMaxScaler()
norm_array = min_max.fit_transform(number_data)

norm_data = pd.DataFrame(norm_array, columns = number_data.columns)

In [75]:
# Применим метод OneHotEncoder к категориальным признакам
from sklearn.preprocessing import OneHotEncoder

oh_enc = OneHotEncoder()
ohe_cats = oh_enc.fit_transform(cat_data).toarray()
ohe_cats = pd.DataFrame(ohe_cats, columns = np.hstack(oh_enc.categories_))

In [76]:
# Выделим целевые признаки и отсеим лишние категориальные
target = ohe_cats[['Adoption', 'Transfer']]
ohe_cats = ohe_cats.drop(['Adoption', 'Transfer', 'Died', 'Disposal', 'Euthanasia', 
                           'Missing', 'Relocate', 'Return to Owner', 'Rto-Adopt'], axis=1)

In [77]:
# Датасет категориальных признаков
ohe_cats.head()

,Cat,Dog,Other,hair_undef,longhair,medium hair,shorthair,large,medium,size_undef,small,col_1,col_2,col_3,Intact,Neutered/Spayed,Unknown
0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [78]:
# Добавим численный нормированный признак к категориальным и создадим датасет, в котором признак 'color' был обработан 
# 1-м способом
X1 = (norm_data.merge(data[['not_domestic']], left_index=True, right_index=True).
                merge(ohe_cats, left_index=True, right_index=True))
X1.head()

,age,not_domestic,Cat,Dog,Other,hair_undef,longhair,medium hair,shorthair,large,medium,size_undef,small,col_1,col_2,col_3,Intact,Neutered/Spayed,Unknown
0,0.001425,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,0.039895,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.039895,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.359930,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.016331,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [79]:
# Удалим из предыдущего датасета признаки окраса и соединим с датасетом цветов и окрасов, полученным из признака 
# 'color', обработанным 2-м способом
X2 = X1.drop(['col_1', 'col_2', 'col_3'], axis=1).merge(color_df, left_index=True, right_index=True)
X2.head()

,age,not_domestic,Cat,Dog,Other,hair_undef,longhair,medium hair,shorthair,large,...,smoke,tabby,tan,tick,tiger,torbie,tortie,tricolor,white,yellow
0,0.001425,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,1,0,0,0,0,0,0,0,0
1,0.039895,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
2,0.039895,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
3,0.359930,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
4,0.016331,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
X1.profile_report()

Из *profile report* видим, что для данных, обработанных 1-м способом, у нас много дубликатов и высокая корреляция признаков **hair_undef** и **Cat**.

In [81]:
X2.profile_report()

Тоже самое для данных, обработанных 2-м способом. Отличие в меньшем количестве дубликатов.

In [82]:
# Удалим дубликаты в датасетах
X1 = X1.drop_duplicates(keep='first')
X2 = X2.drop_duplicates(keep='first')

In [83]:
# Посмотрим на размер
X1.shape, X2.shape

((3107, 19), (15165, 53))

In [84]:
# Целевые признаки изменим в соответствии с данными без дубликатов
target1 = target.iloc[X1.index]
target2 = target.iloc[X2.index]

In [85]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA

In [86]:
def lin_reg_fit(X_train, X_test, y_train, y_test):
    """Обучает модель линейной регрессии по тренировочным и тестовым данным"""
    estimator = LinearRegression()
    estimator.fit(X_train, y_train)
    
    pred_train = estimator.predict(X_train)
    pred_test = estimator.predict(X_test)
    
    train_mae = mean_absolute_error(y_train, pred_train)
    test_mae = mean_absolute_error(y_test, pred_test)
    
    print(f'Train MAE: {np.round(train_mae, 4)}')
    print(f'Test MAE: {np.round(test_mae, 4)}')
    return estimator

In [88]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, target1['Adoption'], test_size=0.2, random_state=13)

print("MAE for 'Adoption' prediction for dataset 1")
lin_reg_fit(X1_train, X1_test, y1_train, y1_test)

MAE for 'Adoption' prediction for dataset 1
Train MAE: 0.2512
Test MAE: 0.2425


LinearRegression()

In [89]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, target1['Transfer'], test_size=0.2, random_state=13)

print("MAE for 'Transfer' prediction for dataset 1")
lin_reg_fit(X1_train, X1_test, y1_train, y1_test)

MAE for 'Transfer' prediction for dataset 1
Train MAE: 0.3661
Test MAE: 0.3712


LinearRegression()

In [90]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, target2['Adoption'], test_size=0.2, random_state=13)

print("MAE for 'Adoption' prediction for dataset 2")
lin_reg_fit(X2_train, X2_test, y2_train, y2_test)

MAE for 'Adoption' prediction for dataset 2
Train MAE: 0.3034
Test MAE: 0.3102


LinearRegression()

In [91]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, target2['Transfer'], test_size=0.2, random_state=13)

print("MAE for 'Transfer' prediction for dataset 2")
lin_reg_fit(X2_train, X2_test, y2_train, y2_test)

MAE for 'Transfer' prediction for dataset 2
Train MAE: 0.352
Test MAE: 0.3482


LinearRegression()

Видим, что точность лучше для данных, в которых был применён 1-й способ обработки признака **color** (датасет 1).

In [101]:
# Применим метод главных компонент для датасета 1
pca_decomp = PCA(n_components=17)
X1_pca = pd.DataFrame(pca_decomp.fit_transform(X1))

X1_pca_train, X1_pca_test, y1_pca_train, y1_pca_test = train_test_split(X1_pca, target1['Adoption'], test_size=0.2, random_state=13)
print("MAE for 'Adoption' prediction for dataset 1 with PCA method")
lin_reg_fit(X1_pca_train, X1_pca_test, y1_pca_train, y1_pca_test)

MAE for 'Adoption' prediction for dataset 1 with PCA method
Train MAE: 0.2508
Test MAE: 0.2434


LinearRegression()

Наименьшая ошибка mae = 0.2434 при числе компонент 17. После применения метода, ошибка стала немного больше.

In [102]:
# Метод главных компонент для датасета 2
pca_decomp = PCA(n_components=27)
X2_pca = pd.DataFrame(pca_decomp.fit_transform(X2))

X2_pca_train, X2_pca_test, y2_pca_train, y2_pca_test = train_test_split(X2_pca, target2['Adoption'], test_size=0.2, random_state=13)
print("MAE for 'Adoption' prediction for dataset 2 with PCA method")
lin_reg_fit(X2_pca_train, X2_pca_test, y2_pca_train, y2_pca_test)

MAE for 'Adoption' prediction for dataset 2 with PCA method
Train MAE: 0.3045
Test MAE: 0.3108


LinearRegression()

Ошибка mae также стала немного больше.  
  
  
Удалим один из сильно коррелирующих признаков и проверим ошибку модели линейной регрессии на датасете 1.

In [103]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1.drop('hair_undef', axis=1), 
                                                         target1['Adoption'], 
                                                         test_size=0.2, 
                                                         random_state=13)

print("MAE for 'Adoption' prediction for dataset 1")
lin_reg_fit(X1_train, X1_test, y1_train, y1_test)

MAE for 'Adoption' prediction for dataset 1
Train MAE: 0.2512
Test MAE: 0.2433


LinearRegression()

Ошибка не стала меньше.  
  
  Построим модель Random Forest.

In [104]:
# Подготовим переменные Adoption и Transfer, а также others для остальных, для Label Encoding.
rfc_target = target1.copy()
rfc_target.loc[(target.Adoption == 0) & (target.Transfer == 0),'others'] = 1
rfc_target.head()

,Adoption,Transfer,others
0,0.0,1.0,NaN
1,0.0,1.0,NaN
3,0.0,1.0,NaN
4,0.0,0.0,1.0
5,0.0,1.0,NaN


In [105]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(rfc_target.idxmax(1))

In [106]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [107]:
X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.2, random_state=13)

rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train)
    
rfc_prediction = rfc.predict(X_test)
    
print(classification_report(y_test, rfc_prediction))
print()
print(f'Random Forest Model Score: {rfc.score(X_test, y_test)}')

              precision    recall  f1-score   support

           0       0.56      0.64      0.60       132
           1       0.65      0.69      0.67       282
           2       0.65      0.54      0.59       208

    accuracy                           0.63       622
   macro avg       0.62      0.62      0.62       622
weighted avg       0.63      0.63      0.63       622


Random Forest Model Score: 0.6286173633440515


Точность модели Random Forest на датасете 1 равна 0.6286.

In [108]:
# Посмотрим на важность признаков датасета 1 для модели Random Forest
headers = list(X1.columns.values)

feature_imp = pd.Series(rfc.feature_importances_,index=headers).sort_values(ascending=False)
feature_imp

age                0.585553
Neutered/Spayed    0.088798
not_domestic       0.050335
Intact             0.031449
Unknown            0.025167
col_1              0.021354
col_3              0.020438
size_undef         0.020402
small              0.019982
large              0.018968
medium             0.018215
col_2              0.017506
Dog                0.013508
Cat                0.013002
shorthair          0.012217
hair_undef         0.011578
Other              0.011371
medium hair        0.010167
longhair           0.009989
dtype: float64

Самый важный признак для модели это возраст. Размер собак и длина шерсти кошек имеют не большую важность, возможно потому что эти признаки только для собак и кошек.  

Построим модель Random Forest для датасета 2. Посмотрим на точность и важность признаков.

In [109]:
rfc_target2 = target2.copy()
rfc_target2.loc[(target.Adoption == 0) & (target.Transfer == 0),'others'] = 1

In [110]:
y2 = le.fit_transform(rfc_target2.idxmax(1))

In [111]:
X_train, X_test, y_train, y_test = train_test_split(X2, y2, test_size=0.2, random_state=13)

rfc.fit(X_train, y_train)
    
rfc_prediction = rfc.predict(X_test)
    
print(classification_report(y_test, rfc_prediction))
print()
print(f'Random Forest Model Score: {rfc.score(X_test, y_test)}')

              precision    recall  f1-score   support

           0       0.63      0.68      0.66       977
           1       0.62      0.65      0.64      1116
           2       0.59      0.51      0.55       940

    accuracy                           0.62      3033
   macro avg       0.61      0.61      0.61      3033
weighted avg       0.61      0.62      0.61      3033


Random Forest Model Score: 0.6158918562479393


In [112]:
headers = list(X2.columns.values)

feature_imp = pd.Series(rfc.feature_importances_,index=headers).sort_values(ascending=False)
feature_imp.head(15)

age                0.437615
Neutered/Spayed    0.086979
Intact             0.048945
white              0.046027
black              0.029520
brown              0.021526
Unknown            0.019934
not_domestic       0.018565
tan                0.016913
size_undef         0.016152
large              0.015802
small              0.014568
medium             0.014206
blue               0.014173
tabby              0.011751
dtype: float64

Видим, что точность стала немного лучше.  
Из изменений: 
- цвета белый, чёрный и коричневый имеют наибольшую важность, может быть из-за того, что их больше
- важность признака неодомашненности животного снизилась  

Для увеличения точности модели, можно попробовать разделить цвета на светлый, тёмный, нейтральный и пёстрый. Также определить размер для всех животных и из признака породы животного выделить как часто животное этой породы встречается.